In [1]:
import numpy as np
import os
import time
folder = "Problems"
time_limit = 30

In [3]:
result = [[] for _ in os.listdir(folder)]
for step, dirs in enumerate(sorted(os.listdir(folder), key = lambda x: int(x))):
    print("Test with number of {} begin!".format(dirs))
    cur_dir = os.path.join(folder, dirs)
    for files in os.listdir(cur_dir):
        start_time = time.time()
        path = os.path.join(cur_dir, files)
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read().split()
            data = list(map(float, content))
            number = int(data[0])
            distances = np.zeros((number,number))
        for i in range(number):
            for j in range(number):
                distances[i][j] = data[i*number+j+1]
        
        ans = np.inf
        ans_route = [""]
        for _ in range(100000):
            if time.time() - start_time > time_limit:
                break
            order = list(np.arange(1, number))
            np.random.shuffle(order)

            route = [0] + order + [0]
            while True:
                if time.time() - start_time > time_limit:
                    break
                improvement = 0
                for i in range(1, number-1):
                    current = distances[route[i-1]][route[i]] + distances[route[i+1]][route[i+2]] - distances[route[i-1]][route[i+2]]
                    for j in range(0, number):
                        if abs(i-j) < 2 :
                            continue  
                        next = distances[route[j]][route[i]] + distances[route[i+1]][route[j+1]] - distances[route[j]][route[j+1]]
                        if current - next > improvement:
                            next_step = (i,j)
                            improvement = current - next
                if improvement == 0:
                    break

                if next_step[1] > next_step[0]+1:
                    route[next_step[0]:next_step[1]+1] = route[next_step[0]+2:next_step[1]+1] + route[next_step[0]:next_step[0]+2]
                    # print(route[next_step[0]+2:next_step[1]+1] + route[next_step[0]:next_step[0]+2])

                else:

                    route[next_step[1]+1:next_step[0]+2] = route[next_step[0]:next_step[0]+2] + route[next_step[1]+1:next_step[0]]

            cur = sum([distances[route[i]][route[i+1]] for i in range(number)])
            if cur < ans:
                ans = cur   
                ans_route = route 

        print("ans is: {}, route is: {}".format(np.round(ans, 2), '->'.join(map(str, ans_route))))
        result[step].append(ans)
    print()
    print()

Test with number of 10 begin!
ans is: 469.78, route is: 0->6->5->4->2->8->3->9->1->7->0
ans is: 449.22, route is: 0->5->4->2->7->9->3->8->6->1->0
ans is: 416.75, route is: 0->3->9->1->7->5->8->6->4->2->0
ans is: 506.28, route is: 0->2->6->4->5->7->1->3->8->9->0
ans is: 518.05, route is: 0->9->2->8->5->1->3->4->7->6->0
ans is: 420.27, route is: 0->3->7->6->5->9->4->8->2->1->0
ans is: 547.28, route is: 0->7->6->1->4->2->8->3->5->9->0
ans is: 484.33, route is: 0->2->6->9->8->1->4->3->5->7->0
ans is: 735.16, route is: 0->1->2->3->7->4->5->8->9->6->0
ans is: 470.02, route is: 0->8->1->6->4->5->3->9->7->2->0


Test with number of 20 begin!
ans is: 746.76, route is: 0->18->9->8->14->5->19->2->16->6->13->1->15->17->10->4->3->11->7->12->0
ans is: 645.11, route is: 0->8->1->18->14->5->16->12->11->9->3->13->17->10->2->7->19->6->4->15->0
ans is: 734.62, route is: 0->7->12->18->16->10->8->13->17->6->14->19->3->11->1->15->2->5->4->9->0
ans is: 455.43, route is: 0->7->5->17->14->4->19->2->3->15->10->

In [5]:
from openpyxl import load_workbook

# 载入已存在的xlsx文件
wb = load_workbook('result.xlsx')

# 选择一个工作表
ws = wb["30s"]

# 写入数据到第A列（你可以更改列名来选择不同的列）
ws[f'E{1}'] = "or_opt"
for i, item in enumerate(result, start=2):  # start=1 表示从第一行开始
    ws[f'E{i}'] = np.mean(item)
# 保存文件
wb.save('result.xlsx')